In [10]:
from ViolaJones import *

In [11]:
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
import numpy as np
import skimage.io as io
import random
import cv2
import os
import matplotlib.patches as patches
from sklearn.metrics import classification_report
from collections import defaultdict

In [12]:
clf2 = ViolaJones.load('model')

In [13]:
all_stages = clf2.stages.copy()
#clf2.stages = all_stages[:2]
clf2.stages = all_stages

In [14]:
def scan_image(img, clf, window_x, window_y, stride_x, stride_y, scale):
    res = []
    img = cv2.resize(img, (img.shape[1] // scale, img.shape[0] // scale))
    x, y = img.shape
    for i in range(0, x - window_x + 1, stride_x):
        for j in range(0, y - window_y + 1, stride_y):
            temp = img[i : i + window_x, j : j + window_y]
            p = clf.classify(temp)
            
            if p == 1:
                res.append((i * scale, j * scale, (i + window_x) * scale, (j + window_y) * scale))
    return res

In [17]:
def get_faces(img, clf):
    window_x = 19
    window_y = 19
    stride_x = 3
    stride_y = 3
    pred = []

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) / 255
    pred = pred + scan_image(gray, clf, window_x, window_y, stride_x, stride_y, 13)

    #img = cv2.resize(img, (150, 150))
    #window_x = 150 // 6
    #window_y = 150 // 5
    return pred #scan_image(img, clf, window_x, window_y, stride_x, stride_y, 2) 


In [19]:
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    #img = cv2.resize(frame, (300, 300))
    img = frame.copy()
    faces_out = get_faces(img, clf2)
    #faces_out = [(200, 100, 200 + 180, 100 + 180)]
    for i, j, x, y in faces_out:
        cv2.rectangle(img, (j,i), (y, x), (255, 0, 0))

    # Display the resulting frame
    img = cv2.flip(img, 1)
    cv2.imshow('frame', img)
    if cv2.waitKey(1) == ord('q'):
        break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [9]:
clf2.save('model', False)